In [ ]:
import tensorflow as tf
import pandas as pd

from stable_baselines.common.policies import MlpLnLstmPolicy
from stable_baselines import PPO2

from tensortrade.strategies import StableBaselinesTradingStrategy
from tensortrade.environments import TradingEnvironment
from tensortrade.rewards import RiskAdjustedReturns
from tensortrade.actions import ManagedRiskOrders
from tensortrade.instruments import Quantity, TradingPair, BTC, USD
from tensortrade.orders.criteria import StopLoss, StopDirection
from tensortrade.wallets import Wallet, Portfolio
from tensortrade.exchanges.simulated import SimulatedExchange
from tensortrade.features.stationarity import LogDifference
from tensortrade.features.scalers import MinMaxNormalizer
from tensortrade.features import FeaturePipeline

WINDOW_SIZE = 1
PRICE_COLUMN = 'close'

normalize = MinMaxNormalizer(inplace=True)
difference = LogDifference(inplace=True)
feature_pipeline = FeaturePipeline(steps=[normalize])

action_scheme = ManagedRiskOrders(pairs=[USD/BTC])
reward_scheme = RiskAdjustedReturns(return_algorithm="sortino")

csv_file = tf.keras.utils.get_file(
    'Coinbase_BTCUSD_1h.csv', 'https://www.cryptodatadownload.com/cdd/Coinbase_BTCUSD_1h.csv')
ohlcv_data = pd.read_csv(csv_file, skiprows=1, index_col="Date")
ohlcv_data.columns = map(str.lower, ohlcv_data.columns)
ohlcv_data = ohlcv_data.rename(columns={'volume btc': 'volume'})

# ohlcv_data = pd.read_csv('./data/updateOHLCdata_master.csv')

exchange = SimulatedExchange(data_frame=ohlcv_data, price_column=PRICE_COLUMN, randomize_time_slices=True)

wallets = [(exchange, USD, 10000), (exchange, BTC, 0)]

portfolio = Portfolio(base_instrument=USD, wallet_tuples=wallets)

environment = TradingEnvironment(exchange=exchange,
                                 portfolio=portfolio,
                                 action_scheme=action_scheme,
                                 reward_scheme=reward_scheme,
                                 feature_pipeline=feature_pipeline,
                                 window_size=WINDOW_SIZE,
                                 observe_wallets=[USD, BTC])

print('Observation Data:')
print(environment.observation_columns)

model = PPO2
policy = MlpLnLstmPolicy
params = { "learning_rate": 1e-5, 'nminibatches': 1 }

strategy = StableBaselinesTradingStrategy(environment=environment,
                                          model=model,
                                          policy=policy,
                                          model_kwargs=params)

strategy.run(steps=50000)

W1210 21:48:35.413784 4491345344 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/tensorforce/core/module.py:31: The name tf.enable_resource_variables is deprecated. Please use tf.compat.v1.enable_resource_variables instead.

W1210 21:48:35.428879 4491345344 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/tensorforce/core/optimizers/tf_optimizer.py:46: The name tf.train.AdadeltaOptimizer is deprecated. Please use tf.compat.v1.train.AdadeltaOptimizer instead.

W1210 21:48:35.429680 4491345344 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/tensorforce/core/optimizers/tf_optimizer.py:47: The name tf.train.AdagradOptimizer is deprecated. Please use tf.compat.v1.train.AdagradOptimizer instead.

W1210 21:48:35.430373 4491345344 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/tensorforce/core/optimizers/tf_optimizer.py:48: The name tf.train.AdamOptimizer is deprecated. Please use tf.compat.v1.train

Observation Data:
['open' 'high' 'low' 'close' 'volume' 'volume usd' 'USD' 'BTC'
 'USD_pending' 'BTC_pending']


/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
I1210 21:49:05.448808 4491345344 trading_environment.py:357] Order: <Order: id=3ed3e0cd-bd66-4346-b2f3-4de937e47d12, status=OrderStatus.FILLED, type=TradeType.MARKET, side=TradeSide.BUY, pair=USD/BTC, quantity=1000.00 USD, pri

Execute:  <Order: id=3ed3e0cd-bd66-4346-b2f3-4de937e47d12, status=OrderStatus.PENDING, type=TradeType.MARKET, side=TradeSide.BUY, pair=USD/BTC, quantity=1000.00 USD, price=11622.61, criteria=None, path_id=3ed3e0cd-bd66-4346-b2f3-4de937e47d12 | Recipes: <Recipe: id=ffe57f68-4b3e-4b46-b3a4-fd23649cb487, type=TradeType.MARKET, pair=USD/BTC, criteria=StopLoss: StopDirection.EITHER | 0.05 (T/P) | 0.05 (S/L)>>
Fill:  <Trade: order_id=3ed3e0cd-bd66-4346-b2f3-4de937e47d12, step=1, base_symbol=USD, quote_symbol=BTC, side=TradeSide.BUY, type=TradeType.MARKET, quantity=997.00 USD, price=11622.61, commission=3.00 USD>
Total traded:  1000.0
Execute:  <Order: id=a594bea0-33d0-4b80-8b4a-c7e1e16dd7f3, status=OrderStatus.PENDING, type=TradeType.MARKET, side=TradeSide.BUY, pair=USD/BTC, quantity=2700.00 USD, price=11463.01, criteria=None, path_id=a594bea0-33d0-4b80-8b4a-c7e1e16dd7f3 | Recipes: <Recipe: id=a01065a1-8fe4-4290-b2bd-cd6eea09b5c7, type=TradeType.MARKET, pair=USD/BTC, criteria=StopLoss: StopD

I1210 21:49:05.666309 4491345344 trading_environment.py:358] Observation: [5.2399000e-05 5.2539800e-05 5.2109400e-05 5.2230000e-05 2.6104000e-06
 1.3657113e-02 9.0720000e-06 1.0000000e-16 1.0000000e-16 7.9322343e-09]
I1210 21:49:05.667562 4491345344 trading_environment.py:359] P/L: 1.0148974676312201
I1210 21:49:05.668993 4491345344 trading_environment.py:360] Reward (6): -0.3447154797170766
I1210 21:49:05.673685 4491345344 trading_environment.py:361] Performance:    BTC  BTC_pending    USD  USD_pending    net_worth step
0  0.0     0.793223  907.2          0.0  9915.148277    5
I1210 21:49:05.711359 4491345344 trading_environment.py:357] Order: <Order: id=77d1d2f8-cc0d-498f-841a-470368d420a9, status=OrderStatus.FILLED, type=TradeType.MARKET, side=TradeSide.BUY, pair=USD/BTC, quantity=907.20 USD, price=11650.91, criteria=None, path_id=77d1d2f8-cc0d-498f-841a-470368d420a9 | Recipes: >
I1210 21:49:05.712564 4491345344 trading_environment.py:358] Observation: [5.25590000e-05 5.27420000e-05

...

I1210 21:50:23.477650 4491345344 trading_environment.py:357] Order: None
I1210 21:50:23.478646 4491345344 trading_environment.py:358] Observation: [3.4120900e-05 3.4165500e-05 3.4116200e-05 3.4138000e-05 1.2327000e-06
 4.2081279e-03 1.0000000e-16 1.0000000e-16 6.3330300e-05 3.1858654e-09]
I1210 21:50:23.479593 4491345344 trading_environment.py:359] P/L: 0.9060449283329999
I1210 21:50:23.480365 4491345344 trading_environment.py:360] Reward (1543): -0.010156217956858513
I1210 21:50:23.483575 4491345344 trading_environment.py:361] Performance:    BTC  BTC_pending  USD  USD_pending    net_worth  step
0  0.0     0.318587  0.0      6333.03  9057.833688  1542
I1210 21:50:23.516417 4491345344 trading_environment.py:357] Order: None
I1210 21:50:23.517353 4491345344 trading_environment.py:358] Observation: [3.4224200e-05 3.4232600e-05 3.4113000e-05 3.4120900e-05 1.5253000e-06
 5.2126169e-03 1.0000000e-16 1.0000000e-16 6.3330300e-05 3.1858654e-09]
I1210 21:50:23.518205 4491345344 trading_environm

I1210 21:50:23.926985 4491345344 trading_environment.py:359] P/L: 0.903082073604
I1210 21:50:23.927700 4491345344 trading_environment.py:360] Reward (1555): -0.010414073794333368
I1210 21:50:23.930866 4491345344 trading_environment.py:361] Performance:    BTC  BTC_pending  USD  USD_pending    net_worth  step
0  0.0     0.318587  0.0      6333.03  9028.319832  1554
I1210 21:50:23.962141 4491345344 trading_environment.py:357] Order: None
I1210 21:50:23.963274 4491345344 trading_environment.py:358] Observation: [3.4281200e-05 3.4281200e-05 3.4210000e-05 3.4214300e-05 1.5590000e-06
 5.3408710e-03 1.0000000e-16 1.0000000e-16 6.3330300e-05 3.1858654e-09]
I1210 21:50:23.964263 4491345344 trading_environment.py:359] P/L: 0.9048167772598499
I1210 21:50:23.964866 4491345344 trading_environment.py:360] Reward (1556): -0.010382105140476418
I1210 21:50:23.967892 4491345344 trading_environment.py:361] Performance:    BTC  BTC_pending  USD  USD_pending    net_worth  step
0  0.0     0.318587  0.0     

I1210 21:50:24.411377 4491345344 trading_environment.py:357] Order: None
I1210 21:50:24.412581 4491345344 trading_environment.py:358] Observation: [3.4488000e-05 3.4536800e-05 3.4440000e-05 3.4457000e-05 2.1211000e-06
 7.3200349e-03 1.0000000e-16 1.0000000e-16 6.3330300e-05 3.1858654e-09]
I1210 21:50:24.413610 4491345344 trading_environment.py:359] P/L: 0.91018719039606
I1210 21:50:24.414443 4491345344 trading_environment.py:360] Reward (1568): -0.00959253724820421
I1210 21:50:24.417904 4491345344 trading_environment.py:361] Performance:    BTC  BTC_pending  USD  USD_pending    net_worth  step
0  0.0     0.318587  0.0      6333.03  9099.068342  1567
I1210 21:50:24.450408 4491345344 trading_environment.py:357] Order: None
I1210 21:50:24.452074 4491345344 trading_environment.py:358] Observation: [3.4579500e-05 3.4589000e-05 3.4458000e-05 3.4488000e-05 2.0869000e-06
 7.2008597e-03 1.0000000e-16 1.0000000e-16 6.3330300e-05 3.1858654e-09]
I1210 21:50:24.453348 4491345344 trading_environment

I1210 21:50:24.866319 4491345344 trading_environment.py:359] P/L: 0.90674613728553
I1210 21:50:24.867043 4491345344 trading_environment.py:360] Reward (1580): -0.009450007558015462
I1210 21:50:24.870002 4491345344 trading_environment.py:361] Performance:    BTC  BTC_pending  USD  USD_pending    net_worth  step
0  0.0     0.318587  0.0      6333.03  9107.271946  1579
I1210 21:50:24.900337 4491345344 trading_environment.py:357] Order: None
I1210 21:50:24.901232 4491345344 trading_environment.py:358] Observation: [3.4399900e-05 3.4420400e-05 3.4333800e-05 3.4336000e-05 8.8530000e-07
 3.0434766e-03 1.0000000e-16 1.0000000e-16 6.3330300e-05 3.1858654e-09]
I1210 21:50:24.902132 4491345344 trading_environment.py:359] P/L: 0.9070927594301699
I1210 21:50:24.902799 4491345344 trading_environment.py:360] Reward (1581): -0.009868070547588332
I1210 21:50:24.905818 4491345344 trading_environment.py:361] Performance:    BTC  BTC_pending  USD  USD_pending    net_worth  step
0  0.0     0.318587  0.0   

I1210 21:50:25.347496 4491345344 trading_environment.py:357] Order: None
I1210 21:50:25.348688 4491345344 trading_environment.py:358] Observation: [3.4325100e-05 3.4334800e-05 3.4287600e-05 3.4313500e-05 1.7975000e-06
 6.1688344e-03 1.0000000e-16 1.0000000e-16 6.3330300e-05 3.1858654e-09]
I1210 21:50:25.349744 4491345344 trading_environment.py:359] P/L: 0.9050044247260199
I1210 21:50:25.350367 4491345344 trading_environment.py:360] Reward (1593): -0.010080855880900203
I1210 21:50:25.353713 4491345344 trading_environment.py:361] Performance:    BTC  BTC_pending  USD  USD_pending    net_worth  step
0  0.0     0.318587  0.0      6333.03  9041.945778  1592
I1210 21:50:25.386274 4491345344 trading_environment.py:357] Order: None
I1210 21:50:25.387374 4491345344 trading_environment.py:358] Observation: [3.4375000e-05 3.4416500e-05 3.4320000e-05 3.4325100e-05 1.9239000e-06
 6.6146649e-03 1.0000000e-16 1.0000000e-16 6.3330300e-05 3.1858654e-09]
I1210 21:50:25.388479 4491345344 trading_environm